In [2]:
import xml.etree.ElementTree as ET
from rdflib import BNode, Literal, Namespace, Graph

# OSM Datei laden
tree = ET.parse('germany-tram.osm')
root = tree.getroot()

g = Graph() # Graphen erstellen

i = 1

# Namespaces deklarieren
wgs = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
geo = Namespace("http://www.opengis.net/ont/geosparql#")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
foaf = Namespace("http://xmlns.com/foaf/0.1/") 
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
starGeo = Namespace("http://blog.stardog.com/geons/")

# Für jeden Node in OSM
for node in root.findall("node"):
    if(node.find("tag[@k='name']") != None):
        name = node.find("tag[@k='name']").get('v') # Wenn Name vorhanden
    elif node.find("tag[@k='amenity']"):
        name=node.find("tag[@k='amenity']").get('v') + str(i) # Wenn Typ vorhanden
        i += 1
    else:
        name="tramstation" + str(i) # Ansonsten
        i +=1
    
    # Blank Nodes erstellen
    locNode = BNode(name)
    geomName = name + "Geom"
    geomNode = BNode(geomName)
    
    # Lat und Lon holen
    lat = node.get('lat')
    lon = node.get('lon')
    
    # Triple schreiben
    g.add( (geomNode, rdf.type, geo.Geometry) )
    g.add( (geomNode, wgs.lat, Literal(lat , datatype=xsd.float)) )
    g.add( (geomNode, wgs.lon, Literal(lon , datatype=xsd.float)) )
    
    g.add( (locNode, rdf.type, starGeo.Location) )
    g.add( (locNode, rdfs.label, Literal(name)) )
    g.add( (locNode, rdf.type, Literal("Straßenbahnstation")) )
    g.add( (locNode, geo.hasGeometry, geomNode) )
    
g.serialize(destination='tram.ttl', format='turtle') # Datei abspeichern
print("Finished") # Signalisierung, dass Programm fertig

Fertsch
